## Notes

"As I explained before: you have to do detection with py-faster-rcnn and then a tracking using deep sort."

## Use mask-r-cnn to get detections in ROI format

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "nba-images")

########################################


class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


"""Create Model and Load Trained Weights"""

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')

# class_names = ['person']

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
print(r['rois'].shape)
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

# Main

In [ ]:
from tools import generate_detections
from deep_sort.detection import Detection

DEEP_SORT_DIR = 'deep_sort'
MASKRCNN_DIR = 'maskrcnn'
TEST_IMG = 'maskrcnn/nba-images/frame_000009.jpg'

CHECKPOINT_PATH = os.path.join(DEEP_SORT_DIR, "checkpoint", "mars-small128.ckpt-68577")

def main():
    # Start up your application, then generate the encoder function.
    encoder = generate_detections.create_box_encoder(CHECKPOINT_PATH)

    # Now, enter the application main loop. Should be something like this:
    for image, detections in my_data_source:
        # - image is a BGR color image
        # - detections is an Nx4 matrix of bounding boxes in
        #   format (x, y, w, h) where (x, y) is the top-left corner
        #   and (w, h) is the extent
        # - features is an Nx128 matrix of corresponding appearance descriptors
        features = encoder(image, detections)
        
        # Generate a list of detections (we simply set the confidence
        # score to 1.0 here).
        detections = [
            Detection(bbox, 1.0, feature) for bbox, feature in
            zip(detections, features)]

        # Call the tracker
        tracker.predict()
        tracker.update(detections)

        # Go on and visualize the results, etc.

In [ ]:
from tools import generate_detections
from deep_sort.detection import Detection

DEEP_SORT_DIR = 'deep_sort'
MASKRCNN_DIR = 'maskrcnn'

CHECKPOINT_PATH = os.path.join(DEEP_SORT_DIR, "checkpoint", "mars-small128.ckpt-68577")

def main():
    # Start up your application, then generate the encoder function.
    encoder = generate_detections.create_box_encoder(CHECKPOINT_PATH)

    # Now, enter the application main loop. Should be something like this:
    for image, detections in my_data_source:
        # - image is a BGR color image
        # - detections is an Nx4 matrix of bounding boxes in
        #   format (x, y, w, h) where (x, y) is the top-left corner
        #   and (w, h) is the extent
        # - features is an Nx128 matrix of corresponding appearance descriptors
        features = encoder(image, detections)
        
        # Generate a list of detections (we simply set the confidence
        # score to 1.0 here).
        detections = [
            Detection(bbox, 1.0, feature) for bbox, feature in
            zip(detections, features)]

        # Call the tracker
        tracker.predict()
        tracker.update(detections)

        # Go on and visualize the results, etc.

# Generate Detections

In [ ]:
import os
import csv
import skvideo.io
import numpy as np
import coco
import utils
import model as modellib
import visualize

# Mask-R-CNN
MASKRCNN_DIR = 'maskrcnn'
MODEL_DIR = os.path.join("logs")
COCO_MODEL_PATH = os.path.join("mask_rcnn_coco.h5")

VIDEO_DIR = 'videos'
VIDEO_FILE = 'transition.mp4'
video = os.path.join(VIDEO_DIR, VIDEO_FILE)

# Save video frames
FRAMES_DIR = 'frames'

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

def get_detections_frame(model, image, frame_idx):
    results = model.detect([image], verbose=0)
    rois = results[0]['rois']
    
    detections = np.zeros([len(rois), 10])
    
    for idx, coord in enumerate(rois):
        detections[idx] = to_mot_format(frame_idx, coord)
    
    return detections


def write_to_csv(csv_file, row_titles = None, new_row = None, new_file = 'no'):
    if new_file == 'yes':
        rwa = 'w'
    else:
        rwa = 'a'
    with open(csv_file, rwa) as csvfile:
        writer = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
        if row_titles:
            writer.writerow(row_titles)
        if new_row:
            writer.writerow(new_row)
    csvfile.close()
    
def to_mot_format(frame_idx, coord):
    """
    Input coordinates: 
    (y1, x1, y2, x2)
    
    Output coordinates: 
    (frame, id, bb_left, bb_top, bb_width, bb_height, -1, -1, -1, -1)
    """
    padding = np.array([-1, -1, -1, -1])
    
    coord = np.insert(coord, 0, frame_idx)
    coord = np.insert(coord, 1, -1)
    coord = np.append(coord, padding)
    
    width = coord[5] - coord[3]
    height = coord[4] - coord[2]
    
    coord[4] = width
    coord[5] = height

    # Rearrange coordinates
    coord = coord[[0, 1, 3, 2, 4, 5, 6, 7, 8, 9]]
    
    return coord

def get_detections_video(video):
    """
    Get ROI detections from video using Mask-R-CNN and save in 
    MOTChallenge format
    """    
    videodata = skvideo.io.vread(video)
    num_frames = len(videodata)
    det_file = open('detections.txt', 'ab')
    
    for idx, frame in enumerate(videodata):
        try:
            print("PROCESSING IMAGE {} / {}".format(idx, num_frames))
            detection = get_detections_frame(model, frame, idx)
            np.savetxt(det_file, detection, delimiter=',', fmt='%1.2f')
            print("DONE")
        except:
            print("FRAME {} NOT PROCESSED".format(idx))

        det_file.flush()
    
    det_file.close()
    
    print("FINISHED!")

# def get_detections_video(video):
#     """
#     Get ROI detections from video using Mask-R-CNN and save in 
#     MOTChallenge format
#     """    
#     camera = cv2.VideoCapture(video)
#     num_frames = int(camera.get(cv2.CAP_PROP_FRAME_COUNT))
#     count = 0
#     det_file = open('detections.txt', 'ab')
    
#     while(camera.isOpened()):
#         success,image = camera.read()
#         if success and count > 360:
#             print("PROCESSING IMAGE {} / {}".format(count, num_frames))
#             detection = get_detections_frame(model, image, count)
#             np.savetxt(det_file, detection, delimiter=',', fmt='%1.2f')
#             print("DONE")

#         else:
#             print("FRAME {} NOT PROCESSED".format(count))

# #         if count == 0:
# #             detections = detection
# #         else:
# #             detections = np.concatenate((detections, detection))
            
#         count += 1
#         det_file.flush()
        
#         if (count > (num_frames)):
#             camera.release()
      
#     det_file.close()
    
#     print("FINISHED!")


    
get_detections_video(video)

In [4]:
import os

from deep_sort.deep_sort import generate_detections
from deep_sort.deep_sort import Detection

DEEP_SORT_DIR = 'deep_sort'
DETECTION_DIR = 'detections'

VIDEO_DIR = 'videos'
VIDEO_FILE = 'transition.mp4'

CHECKPOINT_PATH = os.path.join(DEEP_SORT_DIR, 'checkpoint', 'mars-small128.ckpt-68577')

encoder = generate_detections.create_box_encoder(CHECKPOINT_PATH)
video = os.path.join(VIDEO_DIR, VIDEO_FILE)
name_out = 'generated_detections.txt'
detection_file = 'detections.txt'

def generate_detections_features(encoder, video, name_out, detection_file):

    """Generate detections with features. Modification with video"""
    detections_in = np.loadtxt(detection_file, delimiter=',')
    detections_out = []

    frame_indices = detections_in[:, 0].astype(np.int)
    min_frame_idx = frame_indices.astype(np.int).min()
    max_frame_idx = frame_indices.astype(np.int).max()

    camera = cv2.VideoCapture(video)

    for frame_idx in range(min_frame_idx, max_frame_idx + 1):
        print("Frame %05d/%05d" % (frame_idx, max_frame_idx))
        mask = frame_indices == frame_idx
        rows = detections_in[mask]

        if frame_idx not in frame_indices:
            print("WARNING could not find image for frame %d" % frame_idx)
            continue
        camera.set(cv2.CAP_PROP_POS_FRAMES, frame_idx-1);
        (grabbed, bgr_image) = camera.read()
#         bgr_image = cv2.imread(image_filenames[frame_idx], cv2.IMREAD_COLOR)
        features = encoder(bgr_image, rows[:, 2:6].copy())
        detections_out += [np.r_[(row, feature)] for row, feature
                           in zip(rows, features)]

    # output_filename = os.path.join(output_dir, "%s.npy" % sequence)
    np.save(name_out, np.asarray(detections_out), allow_pickle=False)
    
generate_detections_features(encoder, video, name_out, detection_file)

KeyError: "The name 'net/images:0' refers to a Tensor which does not exist. The operation, 'net/images', does not exist in the graph."

In [ ]:
def tracking(detections_file):
    """ Track objects"""
    frame_idx = 0;
    min_frame_idx = int(detections_file[:, 0].min())
    max_frame_idx = int(detections_file[:, 0].max())
    min_confidence = -1
    min_detection_height = 0
    nms_max_overlap = 0.3
    
    if (display):
        plt.ion()
        fig = plt.figure()

    while True:
        (grabbed, frame) = camera.read()
        frame_idx += 1;
        print("Processing frame %05d" % frame_idx)

        # Load image and generate detections.
        detections = create_detections(detections_file, frame_idx-1, min_detection_height)
        detections = [d for d in detections if d.confidence >= min_confidence]

        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]

        # Update tracker.
        tracker.predict()
        tracker.update(detections)

        # Update visualization.
        if display:
            ax1 = fig.add_subplot(111, aspect='equal')
            # fn = 'mot_benchmark/%s/%s/img1/%06d.jpg'%(phase,seq,frame)
            ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.title(' Tracked Targets')

        # Store results.
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlwh()
            results.append([
                frame_idx, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])

         
            if (display):
                ax1.add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], fill=False, lw=3,ec=colours[track.track_id % 32, :]))
                ax1.set_adjustable('box-forced')
                plt.text(bbox[0], bbox[1], str(track.track_id))

        if(display):
            fig.canvas.flush_events()
            plt.draw()
            ax1.cla()

        if grabbed == False:
            break